In [ ]:
path_consultoria = '/content/drive/MyDrive/Consultoria/Final'

In [ ]:
!cp -r {path_consultoria}/UR5Inv /content/

In [ ]:
!pip install optuna

In [ ]:
from UR5Inv.LTSM import LSTMInversaUR5

In [ ]:
def comparar_modelos_lstm(nombre_modelo, ruta_datos, modelo, ruta_csv=None):
    """
    Crea o actualiza un archivo CSV con métricas de comparación para modelos LSTM.
    Aprovecha las métricas ya calculadas por el modelo sin recalcularlas.

    Parámetros:
        nombre_modelo (str): Nombre identificador del modelo (ej: "Trayectorias_LSTM").
        ruta_datos (str): Ruta al archivo de datos utilizado para entrenar el modelo.
        modelo (LSTMInversaUR5 o LSTMInversaUR5Simple): Instancia del modelo LSTM entrenado.
        ruta_csv (str, opcional): Ruta donde guardar el archivo CSV.
    """
    import pandas as pd
    import numpy as np
    import os
    import time
    import torch

    # Determinar ruta del CSV si no se especifica
    if ruta_csv is None:
        directorio_datos = os.path.dirname(ruta_datos)
        ruta_csv = os.path.join(directorio_datos, "comparacion_modelos.csv")

    # Obtener resultados de la evaluación del modelo (sin recalcular)
    # Asumimos que el modelo ya ha sido evaluado
    results = modelo.evaluar_total() if hasattr(modelo, 'evaluar_total') else None

    # Determinar seq_len según el tipo de modelo
    if hasattr(modelo, 'seq_len'):
        # Para modelos simples
        seq_len = modelo.seq_len
    elif hasattr(modelo, 'study') and modelo.study is not None:
        # Para modelos optimizados con Optuna
        seq_len = modelo.study.best_params.get('seq_len', 20)
    else:
        # Valor por defecto si no se encuentra
        seq_len = 20
        print("Advertencia: No se pudo determinar seq_len, usando valor por defecto (20)")

    # Calcular tiempo de inferencia promedio (1000 muestras)
    tiempos = []
    n_muestras = min(1000, len(modelo.X) - seq_len)
    indices = np.random.choice(n_muestras, min(n_muestras, 500), replace=False)

    for idx in indices:
        # Obtener una secuencia para predecir
        x_seq = modelo.X[idx:idx+seq_len]

        inicio = time.time()
        # Usar el método correcto según el tipo de modelo
        if hasattr(modelo, 'predecir'):
            modelo.predecir(x_seq)
        else:
            modelo._predict_numpy(x_seq)
        fin = time.time()
        tiempos.append(fin - inicio)

    tiempo_inf = np.mean(tiempos) * 1000  # ms

    # Calcular tamaño aproximado del modelo
    if hasattr(modelo, 'modelo_final') and modelo.modelo_final is not None:
        # Contar parámetros del modelo optimizado
        parametros = sum(p.numel() for p in modelo.modelo_final.parameters())
    elif hasattr(modelo, 'model') and modelo.model is not None:
        # Contar parámetros del modelo simple
        parametros = sum(p.numel() for p in modelo.model.parameters())
    else:
        # Estimación si no se puede calcular directamente
        hidden_size = 128  # Valor por defecto
        if hasattr(modelo, 'hidden_size'):
            hidden_size = modelo.hidden_size
        elif hasattr(modelo, 'study') and modelo.study is not None:
            hidden_size = modelo.study.best_params.get('hidden_size', 128)

        num_layers = 2  # Valor por defecto
        if hasattr(modelo, 'num_layers'):
            num_layers = modelo.num_layers
        elif hasattr(modelo, 'study') and modelo.study is not None:
            num_layers = modelo.study.best_params.get('num_layers', 2)

        # Estimación muy básica del número de parámetros
        parametros = (modelo.input_dim * hidden_size * 4 + hidden_size * hidden_size * 4 * (num_layers - 1) +
                     hidden_size * modelo.output_dim + hidden_size + modelo.output_dim)

    # Crear diccionario con las métricas de los resultados existentes
    if results:
        metricas = {
            'modelo': nombre_modelo,
            'MSE_train': results['train']['mse_rad'] if results['train'] else None,
            'MSE_test': results['test']['mse_rad'] if results['test'] else None,
            'MSE_global': results['total']['mse_rad'],
            'tamaño_modelo': parametros,
            'tiempo_inf': tiempo_inf
        }

        # Añadir errores L1 por articulación
        if 'error_por_artic' in results:
            error_por_artic = results['error_por_artic']
            for i, error in enumerate(error_por_artic):
                metricas[f'Errorq{i}L1'] = np.sqrt(error)  # Convertir MSE a error L1 aprox.
    else:
        # Si no tenemos resultados, crear métricas básicas
        metricas = {
            'modelo': nombre_modelo,
            'MSE_train': np.mean(modelo.train_losses[-1]) if hasattr(modelo, 'train_losses') and len(modelo.train_losses) > 0 else None,
            'MSE_test': np.mean(modelo.val_losses[-1]) if hasattr(modelo, 'val_losses') and len(modelo.val_losses) > 0 else None,
            'MSE_global': np.mean(modelo.train_losses[-1]) if hasattr(modelo, 'train_losses') and len(modelo.train_losses) > 0 else None,
            'tamaño_modelo': parametros,
            'tiempo_inf': tiempo_inf
        }

    # Crear o actualizar el archivo CSV
    try:
        df_existente = pd.read_csv(ruta_csv)
        if nombre_modelo in df_existente['modelo'].values:
            df_existente = df_existente[df_existente['modelo'] != nombre_modelo]
        df_nueva_fila = pd.DataFrame([metricas])
        df_final = pd.concat([df_existente, df_nueva_fila], ignore_index=True)
    except (FileNotFoundError, pd.errors.EmptyDataError):
        df_final = pd.DataFrame([metricas])

    df_final.to_csv(ruta_csv, index=False)
    print(f"Comparación actualizada en: {ruta_csv}")

    return df_final

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, Subset
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import time
import os
import optuna
import shutil
from pathlib import Path

## Datos comandos

In [ ]:
!cp -r "Comandos_LSTM" /content/drive/MyDrive/Consultoria/Final


In [ ]:

datos = r"/content/drive/MyDrive/Consultoria/Final/Data/2dos/datos_comandos.csv"

modelo = LSTMInversaUR5(
    ruta_datos=datos,
    test_size=0.25,
    random_state=42,
)
print(f"Datos cargados: {modelo.X.shape[0]} muestras")
print(f"Dimensiones: entrada={modelo.input_dim}, salida={modelo.output_dim}")

# Ejecutamos la optimización con Optuna
print("\nOptimizando hiperparámetros con Optuna...")
modelo.optimizar(n_trials=10)

# Entrenamos el mejor modelo encontrado
print("\nEntrenando el mejor modelo...")
modelo.entrenar_mejor_modelo()

# Visualizaciones
print("\nGenerando gráficos...")
# Gráfico de pérdidas
modelo.graficar_perdidas()

# Evaluación y resumen
print("\nEvaluación del modelo:")
modelo.summary()

# Guardar modelo y resultados
nombre_carpeta = "Comandos_LSTM"
modelo.guardar(nombre_carpeta)

# Guardar métricas comparativas
comparar_modelos_lstm(
    nombre_modelo="Comandos_LSTM",
    ruta_datos=datos,
    modelo=modelo
)

import locale
locale.getpreferredencoding = lambda: 'UTF-8'

!export LANG=C.UTF-8
!export LC_ALL=C.UTF-8
!export LC_CTYPE=C.UTF-8

!cp -r {nombre_carpeta} /content/drive/MyDrive/Consultoria/Final


In [ ]:
nombre_carpeta = "Comandos_LSTM"


import locale
locale.getpreferredencoding = lambda: 'UTF-8'

!export LANG=C.UTF-8
!export LC_ALL=C.UTF-8
!export LC_CTYPE=C.UTF-8

!cp -r {nombre_carpeta} /content/drive/MyDrive/Consultoria/Final

In [ ]:
!cp -r {nombre_carpeta} /content/drive/MyDrive/Consultoria/Final

## Datos Aleatorios

In [ ]:

datos = r"/content/drive/MyDrive/Consultoria/Final/Data/3ros/Aleatorio.csv"

modelo = LSTMInversaUR5(
    ruta_datos=datos,
    test_size=0.25,
    random_state=42,
)
print(f"Datos cargados: {modelo.X.shape[0]} muestras")
print(f"Dimensiones: entrada={modelo.input_dim}, salida={modelo.output_dim}")

# Ejecutamos la optimización con Optuna
print("\nOptimizando hiperparámetros con Optuna...")
modelo.optimizar(n_trials=12)

# Entrenamos el mejor modelo encontrado
print("\nEntrenando el mejor modelo...")
modelo.entrenar_mejor_modelo()

# Visualizaciones
print("\nGenerando gráficos...")
# Gráfico de pérdidas
modelo.graficar_perdidas()

# Evaluación y resumen
print("\nEvaluación del modelo:")
modelo.summary()

# Guardar modelo y resultados
nombre_carpeta = "Aleatorios"
modelo.guardar(nombre_carpeta)

# Guardar métricas comparativas
comparar_modelos_lstm(
    nombre_modelo="Aleatorios",
    ruta_datos=datos,
    modelo=modelo
)




In [ ]:

nombre_carpeta = "Aleatorios_LSTM"

import locale
locale.getpreferredencoding = lambda: 'UTF-8'

!export LANG=C.UTF-8
!export LC_ALL=C.UTF-8
!export LC_CTYPE=C.UTF-8

!cp -r {nombre_carpeta} /content/drive/MyDrive/Consultoria/Final


## Trayectorias

In [ ]:

datos = r"/content/drive/MyDrive/Consultoria/Final/Data/Gen1/openaxes-example-robot-dataset/robot/Complete.csv"

modelo = LSTMInversaUR5(
    ruta_datos=datos,
    test_size=0.25,
    random_state=42,
)
print(f"Datos cargados: {modelo.X.shape[0]} muestras")
print(f"Dimensiones: entrada={modelo.input_dim}, salida={modelo.output_dim}")

# Ejecutamos la optimización con Optuna
print("\nOptimizando hiperparámetros con Optuna...")
modelo.optimizar(n_trials=10)

# Entrenamos el mejor modelo encontrado
print("\nEntrenando el mejor modelo...")
modelo.entrenar_mejor_modelo()

# Visualizaciones
print("\nGenerando gráficos...")
# Gráfico de pérdidas
modelo.graficar_perdidas()

# Evaluación y resumen
print("\nEvaluación del modelo:")
modelo.summary()

# Guardar modelo y resultados
nombre_carpeta = "Trayectorias_LSTM"
modelo.guardar(nombre_carpeta)

# Guardar métricas comparativas
comparar_modelos_lstm(
    nombre_modelo="Trayectorias_LSTM",
    ruta_datos=datos,
    modelo=modelo
)
nombre_carpeta = "Trayectorias_LSTM"
import locale
locale.getpreferredencoding = lambda: 'UTF-8'

!export LANG=C.UTF-8
!export LC_ALL=C.UTF-8
!export LC_CTYPE=C.UTF-8

!cp -r {nombre_carpeta} /content/drive/MyDrive/Consultoria/Final

In [ ]:
nombre_carpeta = "Trayectorias_LSTM"
!cp -r {nombre_carpeta} /content/drive/MyDrive/Consultoria/Final

## PesosVel

In [ ]:

datos = r"/content/drive/MyDrive/Consultoria/Final/Data/Gen2/NISTdata.csv"

modelo = LSTMInversaUR5(
    ruta_datos=datos,
    test_size=0.25,
    random_state=42,
)
print(f"Datos cargados: {modelo.X.shape[0]} muestras")
print(f"Dimensiones: entrada={modelo.input_dim}, salida={modelo.output_dim}")

# Ejecutamos la optimización con Optuna
print("\nOptimizando hiperparámetros con Optuna...")
modelo.optimizar(n_trials=5)

# Entrenamos el mejor modelo encontrado
print("\nEntrenando el mejor modelo...")
modelo.entrenar_mejor_modelo()

# Visualizaciones
print("\nGenerando gráficos...")
# Gráfico de pérdidas
modelo.graficar_perdidas()

# Evaluación y resumen
print("\nEvaluación del modelo:")
modelo.summary()

# Guardar modelo y resultados
nombre_carpeta = "PesosVel"
modelo.guardar(nombre_carpeta)

# Guardar métricas comparativas
comparar_modelos_lstm(
    nombre_modelo="PesosVel_LSTM",
    ruta_datos=datos,
    modelo=modelo
)
nombre_carpeta = "PesosVel_LSTM"
import locale
locale.getpreferredencoding = lambda: 'UTF-8'

!export LANG=C.UTF-8
!export LC_ALL=C.UTF-8
!export LC_CTYPE=C.UTF-8

!cp -r {nombre_carpeta} /content/drive/MyDrive/Consultoria/Final

In [ ]:
!cp -r {nombre_carpeta} /content/drive/MyDrive/Consultoria/Final